# Download de pareceres de patentes da Inova Unicamp

*por Francisco Martellini - setembro 2020*

Este notebook foi construído para uso interno da Agência de Inovação da Unicamp, e possui algumas especificidades para a nomeação dos arquivos gerados e execução no ambiente do Google Colab.

Caso deseje implementá-lo em sua organização, procure pela versão geral (**pyINPI - Patentes.ipynb**) no meu repositório do GitHub, que possui o mesmo código sem as especificidades feitas para a Inova Unicamp.

Você pode encontrar meu repositório com o arquivo em **<https://github.com/frmartellini/inpi>**.

## Instruções

### i) Para fazer a busca na RPI e o download dos pareceres

Execute todas as partes deste arquivo, uma de cada vez, no Google Colab. Clique no ícone da pasta na barra lateral e faça o download do pacote zip com todos os arquivos baixados e do arquivo xlsx com o relatório de pareceres da semana.

**Importante!** Os arquivos dos pareceres para download estão disponíveis somente depois das 15:30h da terça-feira da semana (ou da quarta-feira, quando terça for feriado). Executar antes deste horário irá gerar erro na execução.

### ii) Somente fazer a busca na RPI da semana e gerar um relatório com as informações

Execute no Google Colab, exatamente nesta ordem: **Parte 1**, **Parte 2**, **Parte 3** e **Parte 4**. Clique no ícone da pasta na barra lateral e faça o download do arquivo xlsx. **Não execute as outras partes do programa neste caso!**.

**Importante!** As informações da RPI são disponibilizadas na parte da manhã da terça-feira da semana (ou da quarta-feira, quando terça for feriado), a partir das 08:00h. Neste caso, o programa pode ser usado sem gerar erro na execução.

In [ ]:
#@title PARTE 1: CARREGAR BIBLIOTECAS E FUNÇÕES

#### BIBLIOTECAS NECESSÁRIAS

import requests # método request
import os # comandos de arquivo/diretório
import pandas as pd # pandas dataframe
import numpy as np # numpy
import sys
import shutil # apagar uma ársvore de pastas
import zipfile
import io
from bs4 import BeautifulSoup
import re

####  FUNÇÕES E CLASSES

def excel_report(xls_df,xls_name,sheet_name):
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    # also set the default datetime and date formats.
    writer = pd.ExcelWriter(xls_name + ".xlsx", engine='xlsxwriter', date_format='dd/mm/yyyy')
    # Convert the dataframe to an XlsxWriter Excel object.
    xls_df.to_excel(writer, sheet_name=sheet_name, index=False)
    # Get the xlsxwriter workbook and worksheet objects in order to set the column
    # widths, to make the dates clearer.
    workbook  = writer.book
    worksheet = writer.sheets[sheet_name]
    
    #Iterate through each column and set the width == the max length in that column. A padding length of 2 is also added.
    for i, col in enumerate(xls_df.columns):
        column_len = xls_df[col].astype(str).str.len().max() # find length of column i
        column_len = max(column_len, len(col)) + 2 # Setting the length if the column header is larger than the max column value length
        worksheet.set_column(i, i, column_len) # set the column length
    
    # Close the Pandas Excel writer and output the Excel file
    writer.save()
    print("Relatório concluído.")

def inova_sici(df_sici):
    df_sici.columns = map(str.lower, df_sici.columns) # coloca o nome das colunas em minúsculo
    df_sici.drop(df_sici.columns.difference(['tecnologia','pi','revista','despacho']), axis=1, inplace=True)
    df_sici.columns = ['tecnologia','pi_sici','rpi','cod_despacho']

    # Cria a coluna busca no dataframe
    df_sici['cod_busca'] = df_sici['pi_sici'] # Cria a coluna busca com base na PI
    df_sici['cod_busca'] = df_sici['cod_busca'].str[:-2] # remove o digito identificador
    df_sici['cod_busca'] = df_sici['cod_busca'].str.replace(' ', '') # remove os espaços em branco
    df_sici['cod_busca'] = df_sici['cod_busca'].str.replace('BR', '') # remove o código BR
    
    #Se a tecnologia for antiga (o campo do nome está vazio), substitui tecnologia por pi
    df_sici.tecnologia.replace("", df_sici['pi_sici'], regex=True, inplace=True)
    df_sici.drop(['pi_sici'], axis=1, inplace=True) # remove a coluna pi sici
    
    df_sici['rpi'] = df_sici['rpi'].astype('int')
    df_sici['cod_busca'] = df_sici['cod_busca'].astype('str')
    df_sici['cod_despacho'] = df_sici['cod_despacho'].astype('str')    
    print("Dataframe do SICI construído.")
    
# Classes para acessar as informações da aplicativo web do INPI
class api_inpi:
    def __init__(self, url):
        self.url = url
    
    # Verifica o código de status do arquivo no servidor
    def url_status(self):
        r = requests.head(self.url)
        return r.status_code
    
    # Faz o download do arquivo no pacote zip com a extensão escolhida para a memória
    def xml_extract(self):
        if self.url_status() == 200: # Se o código for 200, procede com download
            #r = requests.get(self.url) # Primeiro faz a requisição do arquivo para o servidor        
            #f_zip = zipfile.ZipFile(io.BytesIO(r.content)) # para simular um objeto do tipo bytes para o arquivo zip
            #f_names = f_zip.namelist() # depois lista todos os arquivos do pacote compactado
            #f_name = [i for i in f_names if ext in i][0] # para localizar o arquivo com a extensão desejada
            #return f_zip.read(f_name) # retornando o conteúdo do arquivo para a memória
            r = requests.get(url, stream=True)
            z = zipfile.ZipFile(io.BytesIO(r.content))
            f_names = z.namelist() # depois lista todos os arquivos do pacote compactado
            for fileName in f_names:
                if fileName.endswith('.xml'): # Check filename endswith xml
                    z.extract(fileName) # Extract a single file from zip
            return fileName # return the name of the file
        if self.url_status() == 302: # Mas se o código for 302, o arquivo ainda não existe
            print("404: O arquivo solicitado não existe.") # então imprime uma mensagem de erro
            
    # Busca os pareceres de patentes para uma instituição por titular ou cotitular
    def busca_titular(self,revista,org,df):
        rpi = revista # número da RPI a ser verificada
        #ext = ".xml" # extensão do arquivo para ser extraído do pacote zip
        f_xml = self.xml_extract() # verifica o status do arquivo no servidor e realiza o download para a memória
        #xml_data = xml_extract().content
        infile = open(f_xml,"r")
        xml_data = infile.read()
        xml_data = re.sub(' +',' ',xml_data) # substitui espaçamentos múltiplos por espaçamentos únicos
        soup = BeautifulSoup(xml_data, "xml") # passing the stored data inside the beautifulsoup parser, storing the returned object
        
        org_soup = soup.find_all('nome-completo', text=lambda x: x is not None and org in x.casefold()) # Busca quantas vezes o nome da(s) organização(ões) apareceu na RPI
        data_revista = soup.find('revista').attrs.get('dataPublicacao') # Extrai a data de publicação
        
        despacho_len = len(org_soup) # quantidade total de registros com os critérios solicitados
        for i in range(despacho_len): # percorre o arquivo da revista em busca dos despachos com os critérios solicitados
            despacho_soup = org_soup[i].find_parent('despacho')
            
            # Procura o número da proteção em toda a revista para verificar repetições
            registro_soup = soup.find_all("numero", text=lambda x: despacho_soup.find('numero').text in x) # Busca quantas vezes o nome da(s) organização(ões) apareceu na RPI
            registro_len = len(registro_soup)
            gestao = ""
            flag = 1
            for j in range(registro_len): # extrai as informações individualmente de casa número de proteção repetido
                registros = registro_soup[j].find_parent('despacho')
        
                # Procura o código de despacho
                cod_despacho = registros.find('codigo')
                if cod_despacho != None:
                    cod_despacho = cod_despacho.text
                    
                # Procura o título da invenção
                titulo = registros.find("titulo", inid="54")
                if titulo != None:
                    titulo = titulo.text
        
                # Procura o número da proteção
                registro = registros.find('numero')
                if registro != None:
                    registro = registro.text
                    
                # Monta o codigo de busca para o servidor
                cod_busca = registro[:-2].replace(' ', '').replace('BR', '')
                                
                # Procura o kind code
                kindcode = registros.find('numero').attrs.get('kindcode')
    
                # Procura a data de depósito
                data_deposito = registros.find('data-deposito')
                if data_deposito != None:
                    data_deposito = data_deposito.text

                # Procura os dados de classificação internacional
                ci_len = len(registros.find_all("classificacao-internacional"))
                ci = ""
                if ci_len > 0:
                    for i in range(ci_len):
                        ci_ano = registros.find_all("classificacao-internacional")[i].text + " (" + registros.find_all("classificacao-internacional")[i].attrs.get('ano') + ")"
                        ci = ci + ci_ano + " | "
                ci = ci[:-3]
            
                # Verifica se é gestão interna
                titular_1 = registros.find("titular", sequencia="1")
                if titular_1 != None and flag == 1:
                    nome_completo = titular_1.find("nome-completo").text
                    if org.lower() not in nome_completo.lower(): # Caso o primeiro titular não seja a organização procurada, a gestão é externa
                        gestao = "Não"
                    if org.lower() in nome_completo.lower(): # Caso o primeiro titular não seja a organização procurada, a gestão é externa
                        gestao = "Sim"
                flag = 2

                # Procura a lista de titulares
                titular_len = len(registros.find_all("titular"))
                titulares = ""
                if titular_len > 0:
                    for i in range(titular_len):
                        titular = registros.find_all("titular")[i].find("nome-completo").text
                        titulares = titulares + titular + " | "
                titulares = titulares[:-3]
    
                # Procura a lista de inventores
                inventor_len = len(registros.find_all("inventor"))
                inventores = ""
                if inventor_len > 0:
                    for i in range(inventor_len):
                        inventor = registros.find_all("inventor")[i].find("nome-completo").text
                        inventores = inventores + inventor + " | "
                inventores = inventores[:-3]
                
                df.loc[len(df)] = [rpi, data_revista,registro, kindcode, cod_despacho,titulo,gestao, titulares, inventores, data_deposito, ci,cod_busca]
        
        df['rpi'] = df['rpi'].astype('int')
        df['cod_busca'] = df['cod_busca'].astype('str')
        df['cod_despacho'] = df['cod_despacho'].astype('str')
        os.remove(f_xml)

def download_parecer(df):
    df.dropna(subset = ['registro'], inplace=True) # remove as linhas onde pi é NaN
    df.dropna(subset = ['inpi_name'], inplace=True) # remove as linhas onde inpi_name é NaN

    df['download'] = True
    
    # Casos em que uma proteção tem dois despachos na mesma revista
    # manter somente o despacho que tem dois documentos
    patentes_df.loc[patentes_df.cod_despacho == "15.11", "download"] = False # duplo com 6.22
    patentes_df.loc[patentes_df.cod_despacho == "8.7", "download"] = False # duplo com 7.5
    
    # Carta patente não se encontra no mesmo servidor que os pareceres
    # e devem ser baixados diretamente no siste do INPI
    patentes_df.loc[patentes_df.cod_despacho == "16.1", "download"] = False
    
    df.loc[df['download'] == False,'download'] = np.nan # Troca o valor False de download por NaN
    df.dropna(subset = ['download'], inplace=True) # remove as linhas onde download é NaN   
    df.drop(['download'], axis=1, inplace=True) # remove a coluna download
    
    # Adicionando flags de contagem procurando arquivos duplicados
    df['dupl'] = df['registro'].duplicated(keep=False)
    df['flag'] = df.groupby('registro').cumcount()
    df.loc[df['dupl'] == False, 'flag'] = "" # flag é vazio se não houver duplicidade

    dict_letters = {'' : '', 0 : '_A',1 : '_B', 2 : '_C', 3 : '_D', 4 : '_E',
                    5 : '_F', 6 : '_G', 7 : '_H', 8 : '_I', 9 : '_J'}
    df['flag']= df['flag'].map(dict_letters) 
    df.drop(['dupl'], axis=1, inplace=True) # remove a coluna dupl
    
    df['f_name'] = df['f_name'] + df['flag'] + '.pdf'
    df.drop(['flag'], axis=1, inplace=True) # remove a coluna flag
    
    i = 1 # contador de posição de linha do dataframe
    for folder_name,inpi_name,f_name,rpi in zip(df['folder_name'],df['inpi_name'],df['f_name'],df['rpi']):
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        url_cam = "http://parecer.inpi.gov.br/download.php?cam=arquivos/RPI/" + str(rpi) + '/' + inpi_name
        f_request = requests.get(url_cam, verify = False)
        path = folder_name + "/" + f_name
        f = open(path, 'wb').write(f_request.content) # faz o download do arquivo
        i+=1

    print("Download dos arquivos concluídos!")
    
print("Bibliotecas carregadas.")

In [ ]:
#@title PARTE 2: DIGITE A RPI PARA A BUSCA

rpi_i =  input("Digite o número da RPI:\n")
rpi_i = int(rpi_i)
rpi_f =  rpi_i # somente implementar a busca em uma série de RPIs caso necessário
rpi_f = int(rpi_f)

In [ ]:
#@title PARTE 3: CARREGAR ARQUIVO DO SICI

# Para uso no Google Colab

from google.colab import files # para uso no google colab
!pip install -q XlsxWriter # para uso no google colab

print('\033[1m' + "\nCarregue o arquivo do SICI com a revista da semana\n" + '\033[0m')

uploaded = files.upload()

for fn in uploaded.keys():
    arquivo=fn
    f_sici = arquivo

In [ ]:
#@title PARTE 4: FAZER A BUSCA NA SEÇÃO DE PATENTES DA RPI

# Cria o dataframe para armazenar as informações da RPI de patentes
column_names = ['rpi','data de publicação','registro','kind code','cod_despacho','título','gestão','titulares','inventores','data de deposito','classificação internacional','cod_busca'] # cria as colunas do dataframe
patentes_df = pd.DataFrame(columns=column_names) # cria o dataframe

rpi = rpi_i
for i in range(rpi_f-rpi_i+1):
    url = "http://revistas.inpi.gov.br/txt/P" + str(rpi) + ".zip"
    
    # Para fazer a busca de patentes
    # usar: api_inpi(url).busca_titular(rpi,"nome da instituição",nome do dataframe)
    #
    #   url = url da seção de patentes da rpi para a revista escolhida (str)
    #   rpi = numero da revista (int)
    #   nome = nome completo da instituição em lower case (str)
    #   dataframe = nome do dataframe para armazenamento
    
    api_inpi(url).busca_titular(rpi,"universidade estadual de campinas",patentes_df)
    rpi+=1

# Mescla com o arquivo do SICI
df_sici = pd.read_html(f_sici, encoding='utf-8', header=0)[0].iloc[:-1] # Converte o arquivo html em um dataframe e remove a última linha
inova_sici(df_sici)

patentes_df = pd.merge(patentes_df, df_sici, left_on=['rpi','cod_busca','cod_despacho'],right_on=['rpi','cod_busca','cod_despacho'],how='left')
patentes_df = patentes_df[['rpi','data de publicação','tecnologia','registro','kind code','cod_despacho','título','gestão','titulares','inventores','data de deposito','classificação internacional','cod_busca']]
del df_sici

patentes_df['tecnologia'] = np.where(patentes_df['tecnologia'] == "_", patentes_df['registro'], patentes_df['tecnologia']) # padrão de nome das tecnologias antigas

# Gera o relatório de pareceres

if rpi_f > rpi_i:
    f_name = "rpi_patentes_" + str(rpi_i) + "-" + str(rpi_f)
    s_name = str(rpi_i) + " - " + str(rpi_f)
if rpi_f <= rpi_i:
    f_name = "rpi_patentes_" + str(rpi_i)
    s_name = str(rpi_i)
    
excel_report(patentes_df.iloc[:,:-1],f_name,s_name)

# Define as colunas de download no padrão do SICI-Inova
patentes_df[['nr_tech', 'nome_tech']] = patentes_df['tecnologia'].str.split('_', 1, expand=True) #split only in the first pattern finded
patentes_df['nr_tech'] = patentes_df['nr_tech'].str.zfill(3) # se o número da tecnologia for menor que 100 adiciona zeros na frente
patentes_df['nr_tech'] = patentes_df['nr_tech'] + "_" # adiciona underline após o número

# Caso das tecnologias antigas - NaN and None values
patentes_df = patentes_df.fillna(value=np.nan) # replace all None values with NaN
patentes_df = patentes_df.fillna('') # replace all NaN with empty spaces

# Cria a nova coluna de nomes da tecnologia
patentes_df.drop(['tecnologia'], axis=1, inplace=True) # remove as colunas tecnologia antiga para criar de novo com o padrão 00X
patentes_df['tecnologia'] = patentes_df['nr_tech'] + patentes_df['nome_tech'] # recria a coluna tecnologia com padrão 00X

# Caso das tecnologias antigas e que não estão no SICI - Nome
patentes_df['tecnologia'] = np.where(patentes_df['tecnologia'] == "", patentes_df['registro'], patentes_df['tecnologia']) # padrão de nome das tecnologias antigas
patentes_df['nr_tech'] = np.where(patentes_df['nome_tech'].isnull(),"",patentes_df['nr_tech']) # if tech name is blank, make tech number blank too
patentes_df['tecnologia'] = np.where(patentes_df.tecnologia.str.endswith(('_')) == True,patentes_df['registro'],patentes_df['tecnologia']) # if the value in tech column endswith "_", replace by the register number
# Replace the nr_tech column with a empty space if is a old inova tech 
patentes_df.loc[patentes_df['nr_tech'].str.startswith("PI"), 'nr_tech'] = ""
patentes_df.loc[patentes_df['nr_tech'].str.startswith("BR"), 'nr_tech'] = ""
patentes_df.loc[patentes_df['nr_tech'].str.startswith("MU"), 'nr_tech'] = ""
patentes_df.loc[patentes_df['nr_tech'].str.startswith("CA"), 'nr_tech'] = ""
patentes_df.loc[patentes_df['nr_tech'].str.startswith("PCT"), 'nr_tech'] = ""

# Cria a coluna com o nome do arquivo para download e a pasta no padrão do SICI-Inova
patentes_df['f_name'] = patentes_df['nr_tech'] + patentes_df['registro'] + "_EXIGENCIA_" + patentes_df['cod_despacho'] + "_" + patentes_df['data de publicação'].str[0:2] + patentes_df['data de publicação'].str[3:5] + patentes_df['data de publicação'].str[8:11]
patentes_df['folder_name'] = "PareceresRPI/" + patentes_df['tecnologia'] + "/" + patentes_df['nr_tech'] + "EXIGENCIAS/" + "E_PAT_RPI" + patentes_df['rpi'].astype(str) + "_" + patentes_df['data de publicação'].str[0:2] + patentes_df['data de publicação'].str[3:5] + patentes_df['data de publicação'].str[8:11]
patentes_df.drop(['nr_tech','nome_tech'], axis=1, inplace=True)

In [ ]:
#@title PARTE 5: FAZER O DOWNLOAD DOS PARECERES DA RPI

column_names = ['inpi_name','rpi','cod_busca'] # cria as colunas do dataframe
df_inpi = pd.DataFrame(columns=column_names) # cria o dataframe

rpi = rpi_i
for i in range(rpi_f-rpi_i+1):
    url = 'https://parecer.inpi.gov.br/arquivos/RPI/' + str(rpi)
    
    # verifica se os arquivos já estão disponíveis no servidor
    if requests.get(url, verify=False).status_code != 200:
        print("Não existem arquivos no servidor para esta revista.")
        sys.exit()
    
    # Monta o dataframe do inpi
    df_temp = pd.read_html(requests.get(url, verify=False).content, encoding='utf-8', header=0)[0].iloc[2:-1]
    df_temp.drop(df_temp.columns.difference(['Name']), axis=1, inplace=True)
    df_temp.columns = ['inpi_name'] # renomeia as colunas do inpi
    df_temp.loc[:,'rpi'] = rpi
    df_temp['cod_busca'] = df_temp['inpi_name'].str[3:-11] # cria a coluna com o código de busca
    df_inpi = df_inpi.append(df_temp) 
    del df_temp
    rpi+=1
    
df_inpi = df_inpi.reset_index(drop=True)

patentes_df = pd.merge(patentes_df, df_inpi, left_on=['rpi','cod_busca'],right_on=['rpi','cod_busca'],how='left') # mescla os dataframe sici e inpi    
patentes_df.drop(['cod_busca'], axis=1, inplace=True)
download_parecer(patentes_df)

In [ ]:
#@title PARTE 6: CRIAR O ARQUIVO ZIP

# Para uso no Google Colab

# download dos arquivos em zip
if rpi_f > rpi_i:
    zip_file = "PareceresRPI_" + str(rpi_i) + "-" + str(rpi_f) + ".zip"
    zip_folder = "PareceresRPI"
if rpi_f <= rpi_i:
    zip_file = "PareceresRPI_" + str(rpi_i) + ".zip"
    zip_folder = "PareceresRPI"

# download dos arquivos em zip
#zip_file = f_revista + ".zip"
#zip_folder = f_revista

# !zip -r /content/file.zip /content/Folder_To_Zip
!zip -r "$zip_file" "$zip_folder" *.xlsx

print("Tudo terminado!")

In [ ]:
#@title PARTE 7: LIMPAR ARQUIVOS DA NUVEM
!rm "$arquivo"
!rm -rf "$zip_folder"